In [102]:
import cv2
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score, confusion_matrix
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

# data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# pretrained models
import torchvision
from torchvision import models, transforms

from functions import *
from classes import *

## Load in metadata

In [15]:
all_metadata = pd.read_csv('../data/all_data_info.csv')

In [16]:
# select artists for the task
artists = ['John Singer Sargent', 'Pablo Picasso', 'Pierre-Auguste Renoir', 'Paul Cezanne', 'Camille Pissarro', 'Paul Gauguin', 
           'Claude Monet', 'Edgar Degas', 'Henri Matisse', 'Vincent van Gogh', 'Childe Hassam', 'Pyotr Konchalovsky', 'Martiros Saryan', 
           'Boris Kustodiev', 'Nicholas Roerich', 'Salvador Dali', 'Alfred Sisley', 'Henri Martin', 'Rene Magritte', 'Konstantin Korovin', 
           'Mary Cassatt', 'Gustave Loiseau', 'John Henry Twachtman', 'Georges Braque', 'Pierre Bonnard', "Georgia O'Keeffe", 
           'Gustave Caillebotte', 'Ilya Mashkov', 'Andy Warhol', 'Theo van Rysselberghe', 'Georges Seurat', 'Edward Hopper', 'Maxime Maufra', 
           'Diego Rivera', 'Henri-Edmond Cross', 'Robert Julian Onderdonk', 'Guy Rose', 'Andre Derain', 'Willard Metcalf', 'Frida Kahlo', 
           'Paul Signac', 'William James Glackens', 'Frantisek Kupka', 'Julian Alden Weir', 'Paul Serusier', 'Max Pechstein', 
           'Victor Borisov-Musatov', 'Armand Guillaumin', 'Spyros Papaloukas', 'Nicolae Darascu', 'Albert Marquet', 'Ion Theodorescu-Sion']

In [17]:
all_metadata['selected'] = all_metadata['artist'].apply(lambda x: x in artists)

In [18]:
metadata = all_metadata[all_metadata['selected'] == True]

In [19]:
df = metadata.drop('selected', axis=1)

In [140]:
len(df)

13894

In [ ]:
# add resizing to 256x256 portion in siamese notebook
# i left out because im unsure what my_train and my_test are exactly
# i have train_256_border - not sure if that includes center crop

In [61]:
df['full_path'] = df.apply(lambda x: f"../data/train_256_border/{x['new_filename']}" if x['in_train'] == True else f"../data/test_256_border/{x['new_filename']}", axis=1)

In [62]:
df.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename,full_path
0,Paul Serusier,1890,genre painting,7099.0,5857.0,9803854.0,wikiart,Cloisonnism,Seaweed Gatherer,train_and_test,False,32996.jpg,../data/test_256_border/32996.jpg
1,Georges Seurat,1884,NaN,6367.0,4226.0,11579390.0,wikipedia,Pointillism,Bathers at Asnières,train_and_test,True,39751.jpg,../data/train_256_border/39751.jpg
2,Paul Signac,NaN,cityscape,5616.0,4312.0,10612858.0,wikiart,Pointillism,View of the Port of Marseilles,train_and_test,True,74221.jpg,../data/train_256_border/74221.jpg
3,Georges Seurat,1884,genre painting,5910.0,4001.0,5330653.0,wikiart,Pointillism,Study for A Sunday on La Grande Jatte,train_and_test,True,31337.jpg,../data/train_256_border/31337.jpg
4,Gustave Caillebotte,1881,genre painting,5164.0,4087.0,3587461.0,wikiart,Impressionism,Rising Road,train_and_test,False,29616.jpg,../data/test_256_border/29616.jpg


## Split data

In [63]:
all_train_df = df[df['in_train'] == True]
all_train_df = all_train_df.reset_index(drop=True)

In [64]:
test_df = df[df['in_train'] == False]
test_df = test_df.reset_index(drop=True)

In [65]:
# train test split on training dataset
train_df = all_train_df.sample(frac=0.8)
val_df = all_train_df[~all_train_df.index.isin(train_df.index)]
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

In [66]:
train_df.to_csv('../data/train.csv', index=None)
val_df.to_csv('../data/val.csv', index=None)

## Create Dataloader

In [67]:
artist_dict = {artist: i for i, artist in enumerate(sorted(df['artist'].unique()))}

In [68]:
train_ds = ArtistDataset(train_df, artist_dict)
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

In [69]:
val_ds = ArtistDataset(val_df, artist_dict)
val_dl = DataLoader(val_ds, batch_size=16, shuffle=True)

## Pretrained AlexNet image classifier model

### LR = 0.01

In [70]:
model = models.alexnet(pretrained=True)

In [71]:
for param in model.parameters():
    param.requires_grad = False

In [72]:
model.classifier[6] = nn.Linear(4096, 52)

In [73]:
params_to_update = []

for param in model.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [76]:
model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=52, bias=True)
)

In [74]:
optimizer = optim.Adam(params_to_update, lr=0.01)
lossFun = nn.CrossEntropyLoss()

In [58]:
def one_pass_classification(model, dataloader, optimizer, lossFun, backwards=True, print_loss=False):
    """
    One training epoch for classification model.
    """
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    avg_accs = []
    for x, y in tqdm(dataloader):
        
        y_pred = model(x)
        loss = lossFun(y_pred, y)
        total_loss += loss.item()
        avg_accs.append(torch.sum(torch.argmax(y_pred, dim=1) == y).item()/len(y))
        
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    avg_loss = total_loss / len(dataloader)
    avg_acc = sum(avg_accs) / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
        print(avg_acc)
    
    return avg_loss, avg_acc

In [75]:
num_epochs = 5
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    print('Epoch: ', epoch)
    
    train_loss, train_acc = one_pass_classification(model, train_dl, optimizer, lossFun)
    train_losses.append(train_loss)
    print('Train loss: ', train_loss)
    print('Train accuracy: ', train_acc)
    
    valid_loss, valid_acc = one_pass_classification(model, val_dl, optimizer, lossFun, backwards=False)
    valid_losses.append(valid_loss)
    print('Valid loss: ', valid_loss)
    print('Valid accuracy: ', valid_acc)

Epoch:  0


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  16.982559205880804
Train accuracy:  0.26107742537313433


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  16.41516054210378
Valid accuracy:  0.314365671641791
Epoch:  1


...

Valid loss:  20.747859915690636
Valid accuracy:  0.38199626865671643
Epoch:  4


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  9.349903146055206
Train accuracy:  0.5904850746268657


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  21.29156914042003
Valid accuracy:  0.38572761194029853


In [79]:
torch.save(model, '../models/classifier_lr01.pt')

### LR = 0.01, WD = 0.01

In [45]:
model_4 = models.alexnet(pretrained=True)

In [46]:
for param in model_4.parameters():
    param.requires_grad = False

In [47]:
model_4.classifier[6] = nn.Linear(4096, 52)

In [48]:
params_to_update = []

for param in model_4.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [49]:
optimizer = optim.Adam(params_to_update, lr=0.01, weight_decay=0.01)
lossFun = nn.CrossEntropyLoss()

In [50]:
num_epochs = 5
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    print('Epoch: ', epoch)
    
    train_loss, train_acc = one_pass_classification(model_4, train_dl, optimizer, lossFun)
    train_losses.append(train_loss)
    print('Train loss: ', train_loss)
    print('Train accuracy: ', train_acc)
    
    valid_loss, valid_acc = one_pass_classification(model_4, val_dl, optimizer, lossFun, backwards=False)
    valid_losses.append(valid_loss)
    print('Valid loss: ', valid_loss)
    print('Valid accuracy: ', valid_acc)

Epoch:  0


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  16.11200719804906
Train accuracy:  0.24871735074626866


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  14.316991791796328
Valid accuracy:  0.3204291044776119
Epoch:  1


...

Valid loss:  15.745638580464606
Valid accuracy:  0.33348880597014924
Epoch:  4


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  14.351306552317604
Train accuracy:  0.37604944029850745


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  16.961115015086843
Valid accuracy:  0.31529850746268656


In [80]:
torch.save(model_4, 'models/classifier_lr01wd01.pt')

### LR = 0.01, WD = 0.001

In [51]:
model_5 = models.alexnet(pretrained=True)

In [52]:
for param in model_5.parameters():
    param.requires_grad = False

In [53]:
model_5.classifier[6] = nn.Linear(4096, 52)

In [54]:
params_to_update = []

for param in model_4.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [55]:
optimizer = optim.Adam(params_to_update, lr=0.01, weight_decay=0.001)
lossFun = nn.CrossEntropyLoss()

In [56]:
num_epochs = 5
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    print('Epoch: ', epoch)
    
    train_loss, train_acc = one_pass_classification(model_5, train_dl, optimizer, lossFun)
    train_losses.append(train_loss)
    print('Train loss: ', train_loss)
    print('Train accuracy: ', train_acc)
    
    valid_loss, valid_acc = one_pass_classification(model_5, val_dl, optimizer, lossFun, backwards=False)
    valid_losses.append(valid_loss)
    print('Valid loss: ', valid_loss)
    print('Valid accuracy: ', valid_acc)

Epoch:  0


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  4.303370224895762
Train accuracy:  0.013759328358208955


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  4.237398869955718
Valid accuracy:  0.018656716417910446
Epoch:  1


...

Valid loss:  4.237398898423607
Valid accuracy:  0.018656716417910446
Epoch:  4


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  4.295518224363897
Train accuracy:  0.01632462686567164


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  4.237398873514204
Valid accuracy:  0.018656716417910446


In [81]:
torch.save(model_5, 'models/classifier_lr01wd001.pt')

### LR = 0.005

In [57]:
model_2 = models.alexnet(pretrained=True)

In [58]:
for param in model_2.parameters():
    param.requires_grad = False

In [59]:
model_2.classifier[6] = nn.Linear(4096, 52)

In [60]:
params_to_update = []

for param in model_2.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [61]:
optimizer = optim.Adam(params_to_update, lr=0.005)
lossFun = nn.CrossEntropyLoss()

In [62]:
num_epochs = 5
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    print('Epoch: ', epoch)
    
    train_loss, train_acc = one_pass_classification(model_2, train_dl, optimizer, lossFun)
    train_losses.append(train_loss)
    print('Train loss: ', train_loss)
    print('Train accuracy: ', train_acc)
    
    valid_loss, valid_acc = one_pass_classification(model_2, val_dl, optimizer, lossFun, backwards=False)
    valid_losses.append(valid_loss)
    print('Valid loss: ', valid_loss)
    print('Valid accuracy: ', valid_acc)

Epoch:  0


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  9.230837301976646
Train accuracy:  0.2888292910447761


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  9.500314895786456
Valid accuracy:  0.3269589552238806
Epoch:  1


...

Valid loss:  11.57838810201901
Valid accuracy:  0.3885261194029851
Epoch:  4


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  4.947100140154362
Train accuracy:  0.6087919776119403


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  12.28867555376309
Valid accuracy:  0.42350746268656714


In [82]:
torch.save(model_2, 'models/classifier_lr005.pt')

### LR = 0.005, WD = 0.1

In [39]:
model_3 = models.alexnet(pretrained=True)

In [40]:
for param in model_3.parameters():
    param.requires_grad = False

In [41]:
model_3.classifier[6] = nn.Linear(4096, 52)

In [42]:
params_to_update = []

for param in model_3.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [43]:
optimizer = optim.Adam(params_to_update, lr=0.005, weight_decay=0.1)
lossFun = nn.CrossEntropyLoss()

In [44]:
num_epochs = 5
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    print('Epoch: ', epoch)
    
    train_loss, train_acc = one_pass_classification(model_3, train_dl, optimizer, lossFun)
    train_losses.append(train_loss)
    print('Train loss: ', train_loss)
    print('Train accuracy: ', train_acc)
    
    valid_loss, valid_acc = one_pass_classification(model_3, val_dl, optimizer, lossFun, backwards=False)
    valid_losses.append(valid_loss)
    print('Valid loss: ', valid_loss)
    print('Valid accuracy: ', valid_acc)

Epoch:  0


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  6.6836837819263115
Train accuracy:  0.23717350746268656


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  6.537798031052547
Valid accuracy:  0.251865671641791
Epoch:  1


...

Valid loss:  6.457734236076696
Valid accuracy:  0.26492537313432835
Epoch:  4


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  6.730166456592617
Train accuracy:  0.26317630597014924


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  6.755594449256783
Valid accuracy:  0.2667910447761194


In [83]:
torch.save(model_3, 'models/classifier_lr005wd1.pt')

### LR = 0.01 with Augmentation

In [110]:
train_ds_aug = ArtistDataset(train_df, artist_dict, augment=True)
train_dl_aug = DataLoader(train_ds, batch_size=16, shuffle=True)

In [111]:
val_ds_aug = ArtistDataset(val_df, artist_dict, True)
val_dl_aug = DataLoader(val_ds, batch_size=16, shuffle=True)

In [112]:
model_6 = models.alexnet(pretrained=True)

In [113]:
for param in model_6.parameters():
    param.requires_grad = False

In [114]:
model_6.classifier[6] = nn.Linear(4096, 52)

In [115]:
params_to_update = []

for param in model_6.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [116]:
optimizer = optim.Adam(params_to_update, lr=0.01)
lossFun = nn.CrossEntropyLoss()

In [119]:
num_epochs = 5
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    print('Epoch: ', epoch)
    
    train_loss, train_acc = one_pass_classification(model_6, train_dl_aug, optimizer, lossFun)
    train_losses.append(train_loss)
    print('Train loss: ', train_loss)
    print('Train accuracy: ', train_acc)
    
    valid_loss, valid_acc = one_pass_classification(model_6, val_dl_aug, optimizer, lossFun, backwards=False)
    valid_losses.append(valid_loss)
    print('Valid loss: ', valid_loss)
    print('Valid accuracy: ', valid_acc)

Epoch:  0


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  16.9611223815982
Train accuracy:  0.26434235074626866


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  15.87655964182384
Valid accuracy:  0.34375
Epoch:  1


...

Valid loss:  19.638409151959774
Valid accuracy:  0.3885261194029851
Epoch:  4


  0%|          | 0/536 [00:00<?, ?it/s]

Train loss:  9.461990278714628
Train accuracy:  0.5843050373134329


  0%|          | 0/134 [00:00<?, ?it/s]

Valid loss:  22.34773578572629
Valid accuracy:  0.37966417910447764


In [ ]:
torch.save(model_6, '../models/classifier_lr01_aug.pt')

## Compare models using validation data

In [131]:
val_pairs_df = pd.read_csv('../data/validation_pairs.csv')

In [132]:
test_pairs_df = pd.read_csv('../data/test_pairs.csv')

In [133]:
val_ds = ArtistPairsDataset(val_pairs_df)
val_dl = DataLoader(val_ds, batch_size=16, shuffle=True)

In [134]:
test_ds = ArtistPairsDataset(test_pairs_df)
test_dl = DataLoader(test_ds, batch_size=16, shuffle=True)

In [95]:
# model LR = 0.01
validation_check(model, val_dl)

  0%|          | 0/371 [00:00<?, ?it/s]

[[3712   38]
 [ 499 1677]]
0.880273406862745


In [74]:
# model LR = 0.005
validation_check(model_2, val_dl)

  0%|          | 0/371 [00:00<?, ?it/s]

[[3658   92]
 [1097 1079]]
0.7356653186274511


In [75]:
# model LR = 0.005 WD = 0.1
validation_check(model_3, val_dl)

  0%|          | 0/371 [00:00<?, ?it/s]

[[3614  136]
 [1688  488]]
0.5939990196078432


In [76]:
# model LR = 0.01 WD = 0.01
validation_check(model_4, val_dl)

  0%|          | 0/371 [00:00<?, ?it/s]

[[3478  272]
 [1952  224]]
0.5152039215686275


In [77]:
# model LR = 0.01 WD = 0.001
validation_check(model_5, val_dl)

  0%|          | 0/371 [00:00<?, ?it/s]

[[3397  353]
 [1908  268]]
0.5145142156862745


In [138]:
# model LR = 0.01 w/augmentation
validation_check(model_6, val_dl)

  0%|          | 0/366 [00:00<?, ?it/s]

[[3687   67]
 [1061 1040]]
0.7385773753118046


Based on the confusion matrices and ROC AUC scores, the model trained with a learning rate of 0.01 had the best performance on validation.

## Run test

In [96]:
validation_check(model, test_dl)

  0%|          | 0/643 [00:00<?, ?it/s]

[[9703  299]
 [ 198   84]]
0.6339891596148854


In [98]:
(9703+84)/(9703+84+299+198)

0.9516725009723843

The classification model is able to accurately predict approximately 95% of pairs, with a 0.634 ROC AUC score.